In [ ]:
!pip install -U langgraph langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.43
    Uninstalling langsmith-0.3.43:
      Successfully uninstalled langsmith-0.3.43


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Load tokenizer and model
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    offload_folder="offload"
)

chat_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    do_sample=True,
    temperature=0.7,
    max_new_tokens=200
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def convert_messages_to_dicts(messages):
    return [{"role": msg.type, "content": msg.content} for msg in messages]

# Define the state TypedDict
class BotState(TypedDict):
    messages: list

# Define system prompt
system_prompt = """You are Bob, a robot designed to execute general tasks based on instructions from humans.\n
Your task is to be a butler at a party.\n\n

Your job is to:\n
1. Greet each guest at the entrance of the party and ask their name and favorite drink.\n
2. Invite them to follow you (close to your left) to a free seat.\n
3. While guiding them, make small talk to find out their interests.\n
4. After they are seated and you’ve discovered all info, give a summary in this format:\n
\t'Guest name: #; Favourite Drink: #; Interest: #.'\n
5. End the conversation by saying 'Goodbye' and telling the guest to seat.\n\n

IMPORTANT:\n
- Don't forget to tell the guest to follow you to the seat, ONLY in the beginning!\n
- Make a minimum of 4 questions when making small talk.\n
- Ask a maximum of 2 questions at a time.\n
- Only summarize in the requested format once you are sure all info was collected.
"""

def handle_guest_input(state: BotState) -> BotState:
    input_text = tokenizer.apply_chat_template(
        convert_messages_to_dicts(state["messages"]),
        tokenize=False,
        add_generation_prompt=True
    )
    output = chat_pipeline(input_text)[0]["generated_text"].strip()
    state["messages"].append(AIMessage(content=output))
    return state

def is_done(state: BotState) -> str:
    last_msg = state["messages"][-1].content.lower()
    return END if "goodbye" in last_msg else "continue"

builder = StateGraph(BotState)
builder.set_entry_point("chat")
builder.add_node("chat", handle_guest_input)
builder.add_conditional_edges("chat", is_done)

graph = builder.compile()

initial_state = {"messages": [
    SystemMessage(content=system_prompt),
    AIMessage(content="Hello, I am Bob and I will be your butler for the night! Welcome to the party. May I know your name and your favorite drink?")
]}

state = initial_state
print("Bob:", state["messages"][-1].content)

while True:
  guest_input = input("Guest: ")
  state["messages"].append(HumanMessage(content=guest_input))
  state = graph.invoke(state)
  bob_reply = state["messages"][-1].content
  print("Bob:", bob_reply)
  if "goodbye" in bob_reply.lower():
      break


In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

greet_guest_prompt = """You are Bob, a robot designed to execute general tasks based on instructions from humans.\n
Your task is to be a butler at a party.\n\n

Your job is to:\n
1. Greet each guest at the entrance of the party and ask their name and favorite drink.\n
"""

guide_and_small_talk_prompt = """You are Bob, a robot designed to execute general tasks based on instructions from humans.\n
Your task is to be a butler at a party.\n\n

Your job is to:\n
1. Invite them to follow you (close to your left) to a free seat.\n
2. While guiding them, make small talk to find out their interests.\n

IMPORTANT:\n
- Make a minimum of 4 questions when making small talk.\n
- Ask a maximum of 2 questions at a time.
"""

summarize_and_goodbye_prompt = """You are Bob, a robot designed to execute general tasks based on instructions from humans.\n
Your task is to be a butler at a party.\n\n

Your job is to:\n
1. After they are seated and you’ve discovered all info, give a summary in this format:\n
\t'Guest name: #; Favourite Drink: #; Interest: #.'\n
2. End the conversation by saying 'Goodbye' and telling the guest to seat.\n\n

IMPORTANT:\n
- Only summarize in the requested format once you are sure all info was collected.
"""

def greet_guest(state: State) -> State:
    # Append system prompt for this node
    messages = state["messages"].copy()
    messages.append(SystemMessage(content=greet_guest_prompt))

    # Prepare the conversation for the model (convert messages to text inputs)
    conversation_text = ""
    for msg in messages:
        role = "Bob" if isinstance(msg, AIMessage) else "Guest"
        conversation_text += f"{role}: {msg.content}\n"
    conversation_text += "Bob: "  # model should generate after this

    # Generate the LLM response
    generated = chat_pipeline(conversation_text)[0]["generated_text"]
    # Extract Bob's reply after the last "Bob: "
    reply = generated[len(conversation_text):].strip()

    # Append LLM reply to messages
    messages.append(AIMessage(content=reply))

    # Return updated state
    return {"messages": messages}

def guide_and_small_talk(state: State) -> State:
    messages = state["messages"].copy()
    messages.append(SystemMessage(content=guide_and_small_talk_prompt))

    conversation_text = ""
    for msg in messages:
        role = "Bob" if isinstance(msg, AIMessage) else "Guest"
        conversation_text += f"{role}: {msg.content}\n"
    conversation_text += "Bob: "

    generated = chat_pipeline(conversation_text)[0]["generated_text"]
    reply = generated[len(conversation_text):].strip()

    messages.append(AIMessage(content=reply))
    return {"messages": messages}

def summarize_and_goodbye(state: State) -> State:
    messages = state["messages"].copy()
    messages.append(SystemMessage(content=summarize_and_goodbye_prompt))

    conversation_text = ""
    for msg in messages:
        role = "Bob" if isinstance(msg, AIMessage) else "Guest"
        conversation_text += f"{role}: {msg.content}\n"
    conversation_text += "Bob: "

    generated = chat_pipeline(conversation_text)[0]["generated_text"]
    reply = generated[len(conversation_text):].strip()

    messages.append(AIMessage(content=reply))
    return {"messages": messages}

graph_builder = StateGraph(State)
graph_builder.add_node("GreetGuest", greet_guest)
graph_builder.add_node("GuideAndSmallTalk", guide_and_small_talk)
graph_builder.add_node("SummarizeAndGoodbye", summarize_and_goodbye)

graph_builder.add_edge(START, "GreetGuest")
graph_builder.add_edge("GreetGuest", "GuideAndSmallTalk")
graph_builder.add_edge("GuideAndSmallTalk", "SummarizeAndGoodbye")
graph_builder.add_edge("SummarizeAndGoodbye", END)

graph = graph_builder.compile()

initial_state = {"messages": [
    SystemMessage(content=greet_guest_prompt),
    AIMessage(content="Hello, I am Bob and I will be your butler for the night! Welcome to the party. May I know your name and your favorite drink?")
]}

state = initial_state
print("Bob:", state["messages"][-1].content)

while True:
  guest_input = input("Guest: ")
  state["messages"].append(HumanMessage(content=guest_input))
  state = graph.invoke(state)
  bob_reply = state["messages"][-1].content
  print("Bob:", bob_reply)

Bob: Hello, I am Bob and I will be your butler for the night! Welcome to the party. May I know your name and your favorite drink?
Guest: My name is kevin and i like water


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Bob: 
Guest: What?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
